In [ ]:
# インポート
import os
import glob
import argparse
import numpy as np
import zipfile
import time
import datetime

import sklearn  
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import cv2

import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D
from tensorflow.keras import optimizers

# from common_flags import FLAGS
from tensorflow.keras.utils import plot_model, to_categorical

#package modul version 表示

print("tensorflow: ", tf.__version__)
print("numpy: ", np.__version__)
print("pandas: ", pd.__version__)
print("sklearn (for train_test_split): ", sklearn.__version__)
print("matplotlib (for pyplot): ", matplotlib.__version__)
print("opencv (cv2): ", cv2.__version__)
print("tensorflow-gpu: ", tf.test.is_built_with_cuda())

!python -V

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # メモリ成長を有効にする
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [ ]:
# 訓練時の画像サイズ
image_width = 160         
image_height = 120

# 訓練画像を読み込む関数
def load_images(input_dir, X, Y, Z):
    img_list = pd.read_csv(input_dir + '/list2.txt', header=None, delim_whitespace=True)
    image_path = input_dir + '/images/' + img_list[0]

    # 画像データの読み込み
    for index, file_name in enumerate(image_path):
        image = cv2.imread(file_name)
        resize_image = cv2.resize(image, (image_width, image_height))
        data = np.asarray(resize_image)
        X.append(data)
        Y.append(img_list[1][index])
        Z.append(img_list[2][index])
    return X, Y, Z

In [ ]:
def cnn(input_shape):
    drop = 0.5

    # 入力テンソルの定義
    img_input = Input(shape=(input_shape), name='input')

    # 畳み込みとプーリング層
    x1 = Conv2D(32, (3, 3), strides=(2,2), activation='relu')(img_input)
    x2 = MaxPooling2D((2, 2))(x1)
    x3 = Conv2D(64, (3, 3), strides=(2,2), activation='relu')(x2)
    x4 = MaxPooling2D((2, 2))(x3)
    x5 = Conv2D(128, (3, 3), strides=(2,2), activation='relu')(x4)
    x6 = MaxPooling2D((2, 2))(x5)

    # フラットニングと全結合層
    x7 = Flatten(name='flattened')(x6)
    x8 = Dense(128, activation='relu')(x7)

    ##出力層
    output_x = Dense(1, activation='linear', name='output_x')(x8)
    output_y = Dense(1, activation='linear', name='output_y')(x8)

    # モデルの定義
    model = Model(inputs=img_input, outputs=[output_x, output_y])

    return model


In [ ]:
# メイン関数の定義　callbackがあるバージョン
def _main():
    X = []
    Y = []
    Z = []
    X, Y, Z =load_images("./1_image_data", X, Y, Z) # 1_images_dataの読み込み
    X, Y, Z=load_images("./2_image_data", X, Y, Z) # 2_images_dataの読み込み障害物を含むデータ
    X, Y, Z=load_images("./3_image_data", X, Y, Z) # 3_images_dataの読み込み
    X, Y, Z=load_images("./4_image_data", X, Y, Z) # 4_images_dataの読み込み

    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(Z)

    X = X.astype('float32')
    X = X / 255.0          #画像データを0から1の範囲に変換 これを正規化という画像値は0~255の値をとる
    Y = Y.astype('float32')# 正解データ（steer）の微調整
    Z = Z.astype('float32')# 正解データ（thorottle）の微調整
    # 学習用データとテストデータ
    X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(X, Y, Z, test_size=0.20) #0.20なら20%がテストデータ
    shape = X_train.shape[1:]
    model = cnn(shape)
    # ハイパーパラメータ
    decay = 1e-6
    lr = 1e-3
    batch_size = 64
    epochs = 500
    loss_weights = [1,1] #lossの重要度の振り分けを決める
    #コールバックの設定
    early_stpping_callback = tf.keras.callbacks.EarlyStopping(
        monitor ='val_loss',
        patience = 20,
        verbose = 1,
        mode = 'min',
        restore_best_weights = True
    )
    # コンパイル
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr, decay = decay),
                loss={'output_x': tf.keras.losses.huber,
                      'output_y': tf.keras.losses.huber},
                loss_weights = loss_weights,
                )
    # 訓練
    history = model.fit(x =  X_train,
                        y = {'output_x': Y_train, 'output_y': Z_train},
                        batch_size = batch_size,
                        epochs = epochs,
                        verbose = 1,
                        validation_data = ({'input': X_test}, {'output_x': Y_test, 'output_y': Z_test}),
                        callbacks = [early_stpping_callback],
                       )

    # モデルの保存
    model.save('./Stopping_model.h5')
    # 評価 & 評価結果出力
    #print(model.evaluate(X_test, {'output1': Y_test, 'output2': Z_test}))
    # モデルの図示化
    #plot_model(model, to_file='./17Stopping_model.png', show_shapes=True)
    # 評価 & 評価結果出力
    #print(model.evaluate(X_test, {'output1': Y_test, 'output2': Z_test}))
    plot_history_loss(history)# show result graph

In [ ]:
#グラフを一つにまとめた
def plot_history_loss(history):
    plt.figure(figsize=(10, 6))

    # ステアリング損失（トレーニングとバリデーション）
    plt.plot(history.history['output_x_loss'], label='x loss for training', color='blue')
    plt.plot(history.history['val_output_x_loss'], label='x loss for validation', color='cyan')

    # スロットル損失（トレーニングとバリデーション）
    plt.plot(history.history['output_y_loss'], label='y loss for training', color='red')
    plt.plot(history.history['val_output_y_loss'], label='y loss for validation', color='magenta')

    # グラフの設定
    plt.xlabel('Epochs', fontsize=22)
    plt.ylabel('Loss', fontsize=22)
    plt.legend(loc='best', fontsize=22)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)

    plt.savefig('./combined_loss.png')
    plt.show()
    plt.close()

In [ ]:
#mainプログラムの実行
_main()